In [2]:

import numpy as np  # Importing NumPy for numerical operations and array manipulations
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting graphs and visualizations
import seaborn as sns  # Importing Seaborn for statistical data visualization, built on top of Matplotlib
import tensorflow as tf
import tensorflow as tf
  # Importing TensorFlow for building and training machine learning models
from tensorflow import keras  # Importing Keras, a high-level API for TensorFlow, to simplify model building
from tensorflow.keras import Layer  # Importing Layer class for creating custom layers in Keras
from tensorflow.keras.models import Sequential  # Importing Sequential model for building neural networks layer-by-layer
from tensorflow.keras.layers import Rescaling , GlobalAveragePooling2D
from tensorflow.keras import layers, optimizers, callbacks  # Importing various modules for layers, optimizers, and callbacks in Keras
from sklearn.utils.class_weight import compute_class_weight  # Importing function to compute class weights for imbalanced datasets
from tensorflow.keras.applications import EfficientNetV2B2  # Importing EfficientNetV2S model for transfer learning
from sklearn.metrics import confusion_matrix, classification_report  # Importing functions to evaluate model performance
import gradio as gr  # Importing Gradio for creating interactive web interfaces for machine learning models

In [3]:
import zipfile

# Path to your zip file
zip_path = "dataset.zip"  # <-- Change this if your zip name is different
extract_to = "dataset"

# Unzipping process
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Dataset unzipped successfully into:", extract_to)


✅ Dataset unzipped successfully into: dataset


In [4]:
dataset_dir = r"dataset/dataset"

image_size = (124, 124)
batch_size = 32
seed = 42

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    shuffle=True,
    image_size=image_size,
    batch_size=batch_size
)

# Save class names BEFORE applying .map or .cache
class_names = train_ds.class_names
print("Class names:", class_names)
print("Number of classes:", len(class_names))


Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Class names: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Number of classes: 6


In [23]:
print(class_names)
print(len(class_names))


['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
6


In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    shuffle=True,
    image_size=image_size,
    batch_size=batch_size
)

# ✅ Store class names before .map()
class_names = train_ds.class_names


Found 2527 files belonging to 6 classes.
Using 2022 files for training.


In [7]:
class_names = train_ds.class_names  # Store this early


In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    shuffle = True,
    image_size=image_size,
    batch_size=batch_size
)

from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))


Found 2527 files belonging to 6 classes.
Using 2022 files for training.


In [9]:

# ✅ Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))


In [10]:


import gdown
import zipfile
import os

# Download from Google Drive
file_id = "1dEew1Hv-IWv9KFNkMw0c8ncYY_l8d_Dz"
url = f"https://drive.google.com/uc?id={file_id}"
output = "garbage-dataset.zip"

gdown.download(url, output, quiet=False)

# Unzip the dataset
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("garbage-dataset")

print("✅ Dataset downloaded and extracted.")


Downloading...
From (original): https://drive.google.com/uc?id=1dEew1Hv-IWv9KFNkMw0c8ncYY_l8d_Dz
From (redirected): https://drive.google.com/uc?id=1dEew1Hv-IWv9KFNkMw0c8ncYY_l8d_Dz&confirm=t&uuid=59f78fe6-8c57-4105-877e-8ee05c87759b
To: c:\Users\sneha sucharitha\Documents\garbage-classification-ai\garbage-dataset.zip
100%|██████████| 42.9M/42.9M [00:06<00:00, 6.42MB/s]


✅ Dataset downloaded and extracted.


In [11]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    shuffle = True,
    image_size=image_size,
    batch_size=batch_size
)
val_class= val_ds.class_names

from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))  # Add this line if val_ds exists

Found 2527 files belonging to 6 classes.
Using 505 files for validation.


In [17]:
# Get the total number of batches in the validation dataset
val_batches = tf.data.experimental.cardinality(val_ds)  

# Split the validation dataset into two equal parts:
# First half becomes the test dataset
test_ds = val_ds.take(val_batches // 2)  

# Second half remains as the validation dataset
val_dat = val_ds.skip(val_batches // 2)  

# Optimize test dataset by caching and prefetching to improve performance
test_ds_eval = test_ds.cache().prefetch(tf.data.AUTOTUNE)  

In [24]:
print(train_ds.class_names)
print(val_class)
print(len(train_ds.class_names))

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
6


In [25]:
class_names = train_ds.class_names  # Save once
print("Classes:", class_names)


Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [26]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(12):
    ax = plt.subplot(4, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")

In [27]:
def count_distribution(dataset, class_names):
    total = 0
    counts = {name: 0 for name in class_names}
    
    for _, labels in dataset:
        for label in labels.numpy():
            class_name = class_names[label]
            counts[class_name] += 1
            total += 1

    for k in counts:
        counts[k] = round((counts[k] / total) * 100, 2)  # Convert to percentage
    return counts


In [19]:
# Function to plot class distribution
def simple_bar_plot(dist, title):
    plt.bar(dist.keys(), dist.values(), color='cornflowerblue')
    plt.title(title)
    plt.ylabel('Percentage (%)')
    plt.xticks(rotation=45)
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.show()

In [28]:
class_names = train_ds.class_names

# Get class distributions
train_dist = count_distribution(train_ds, class_names)
val_dist = count_distribution(val_ds, class_names)
test_dist = count_distribution(test_ds, class_names)
overall_dist = {}
for k in class_names:
    overall_dist[k] = round((train_dist[k] + val_dist[k]) / 2, 2)

print(train_dist)
print(val_dist)
print(test_dist)
print(overall_dist)

{'cardboard': 16.52, 'glass': 19.73, 'metal': 15.92, 'paper': 23.29, 'plastic': 19.44, 'trash': 5.09}
{'cardboard': 13.66, 'glass': 20.2, 'metal': 17.43, 'paper': 24.36, 'plastic': 17.62, 'trash': 6.73}
{'cardboard': 14.06, 'glass': 17.58, 'metal': 16.8, 'paper': 25.0, 'plastic': 19.14, 'trash': 7.42}
{'cardboard': 15.09, 'glass': 19.96, 'metal': 16.68, 'paper': 23.82, 'plastic': 18.53, 'trash': 5.91}


In [29]:
# Show visualizations
simple_bar_plot(train_dist, "Training Set Class Distribution (%)")
simple_bar_plot(val_dist, "Validation Set Class Distribution (%)")
simple_bar_plot(test_dist, "Test Set Class Distribution (%)")
simple_bar_plot(overall_dist, "Overall Class Distribution (%)")

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 12 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [30]:


# Count class occurrences and prepare label list
class_counts = {i: 0 for i in range(len(class_names))}
all_labels = []

for images, labels in train_ds:
    for label in labels.numpy():
        class_counts[label] += 1
        all_labels.append(label)

# Compute class weights (index aligned)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(class_names)),
    y=all_labels
)

# Create dictionary mapping class index to weight
class_weights = {i: w for i, w in enumerate(class_weights_array)}


In [31]:

# ✅ Optional: print results
print("Class Counts:", class_counts)
print("Class Weights:", class_weights)


Class Counts: {0: 334, 1: 399, 2: 322, 3: 471, 4: 393, 5: 103}
Class Weights: {0: np.float64(1.0089820359281436), 1: np.float64(0.8446115288220551), 2: np.float64(1.046583850931677), 3: np.float64(0.7154989384288747), 4: np.float64(0.8575063613231552), 5: np.float64(3.2718446601941746)}


In [32]:
#  Define data augmentation pipeline
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

In [33]:
#  Load the pretrained MobileNetV3Small model (without the top classification layer)
base_model = EfficientNetV2B2(include_top=False, input_shape=(124, 124, 3),include_preprocessing=True, weights='imagenet')


#  Freeze early layers (to retain general pretrained features)
base_model.trainable = True
for layer in base_model.layers[:100]:  # You can adjust this number
    layer.trainable = False



In [34]:

#  Build the final model
model = Sequential([
    layers.Input(shape=(124, 124, 3)),
    data_augmentation,
    base_model,
    GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(6, activation='softmax')  # Change to your number of classes
])


In [35]:
# ⚙️ Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [36]:
# Define an EarlyStopping callback to stop training when validation loss stops improving
early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',            # Metric to monitor (validation loss here)
    patience=3,                   # Number of epochs to wait after last improvement before stopping
    restore_best_weights=True     # After stopping, restore the model weights from the epoch with the best val_loss
)


In [37]:
# Set the number of epochs to train the model
epochs = 15  # Number of times the model will go through the entire dataset

# Train the model using the fit function
history = model.fit(
    train_ds,                # Training dataset used to adjust model weights
    validation_data=val_ds,   # Validation dataset to monitor performance on unseen data
    epochs=epochs,           # Number of training cycles, referencing the variable set earlier
    class_weight=class_weights,  # Handles class imbalances by assigning appropriate weights
    batch_size=32,           # Number of samples processed in each training step
    callbacks=[early]        # Implements early stopping to prevent unnecessary training
)

Epoch 1/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.2717 - loss: 1.7325 - val_accuracy: 0.6158 - val_loss: 1.2055
Epoch 2/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.6754 - loss: 1.0657 - val_accuracy: 0.7426 - val_loss: 0.8390
Epoch 3/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 580ms/step - accuracy: 0.7880 - loss: 0.6906 - val_accuracy: 0.8158 - val_loss: 0.6280
Epoch 4/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 579ms/step - accuracy: 0.8428 - loss: 0.4920 - val_accuracy: 0.8317 - val_loss: 0.5309
Epoch 5/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 577ms/step - accuracy: 0.8891 - loss: 0.3688 - val_accuracy: 0.8554 - val_loss: 0.4470
Epoch 6/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 41s 641ms/step - accuracy: 0.9350 - loss: 0.2504 - val_accuracy: 0.8653 - val_loss: 0.4071
Epoch 7/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 654ms/step - accuracy: 0.9276 - loss: 0.2283 - val_accuracy: 0.8772 - val_loss: 0.3984
Epoch 8/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 653ms/step - accuracy: 0.9341 - loss: 0.1902 - val_accuracy:

In [38]:
# 📉 Reduce learning rate when val_loss plateaus
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-6,
    verbose=1
)


In [39]:
model.save("Efficientnetv2b2.keras", save_format="keras")


In [40]:
# 🔧 Fine-tuning the top 20 layers of EfficientNetV2B2
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

fine_tune_epochs = 5
total_epochs = epochs + fine_tune_epochs

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    class_weight=class_weights,
    callbacks=[early, reduce_lr]
)


Epoch 15/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 40s 431ms/step - accuracy: 0.9862 - loss: 0.0589 - val_accuracy: 0.9188 - val_loss: 0.2861 - learning_rate: 1.0000e-05
Epoch 16/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 24s 371ms/step - accuracy: 0.9828 - loss: 0.0631 - val_accuracy: 0.9149 - val_loss: 0.2918 - learning_rate: 1.0000e-05
Epoch 17/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 0.9863 - loss: 0.0670
Epoch 17: ReduceLROnPlateau reducing learning rate to 1.9999999494757505e-06.
64/64 ━━━━━━━━━━━━━━━━━━━━ 23s 357ms/step - accuracy: 0.9863 - loss: 0.0671 - val_accuracy: 0.9168 - val_loss: 0.2909 - learning_rate: 1.0000e-05
Epoch 18/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 22s 345ms/step - accuracy: 0.9854 - loss: 0.0585 - val_accuracy: 0.9188 - val_loss: 0.2889 - learning_rate: 2.0000e-06


In [41]:
# 📝 Summary (optional but useful)
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ (None, 124, 124, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b2 (Functional)  │ (None, 4, 4, 1408)     │     8,769,374 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1408)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │         8,454 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,719,898 (40.89 MB)

 Trainable params: 971,034 (3.70 MB)

 Non-trainable params: 7,806,794 (29.78 MB)

 Optimizer params: 1,942,070 (7.41 MB)

In [42]:
base_model.summary() # Print the architecture summary of the base model

Model: "efficientnetv2-b2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 124, 124,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 124, 124,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 124, 124,  │          0 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 62, 62,    │        864 │ normalization[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 62, 62,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 62, 62,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 62, 62,    │      4,608 │ stem_activation[… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 62, 62,    │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 62, 62,    │          0 │ block1a_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 62, 62,    │      2,304 │ block1a_project_… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 62, 62,    │         64 │ block1b_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 62, 62,    │          0 │ block1b_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 62, 62,    │          0 │ block1b_project_… │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 62, 62,    │          0 │ block1b_drop[0][… │
│                     │ 16)               │            │ block1a_project_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 31, 31,    │      9,216 │ block1b_add[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 31, 31,    │        256 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 31, 31,    │          0 │ block2a_expand_b

 Total params: 8,769,374 (33.45 MB)

 Trainable params: 962,580 (3.67 MB)

 Non-trainable params: 7,806,794 (29.78 MB)

In [43]:

acc = history.history['accuracy']          # Extract training accuracy from history
val_acc = history.history['val_accuracy']  # Extract validation accuracy from history
loss = history.history['loss']             # Extract training loss from history
val_loss = history.history['val_loss']     # Extract validation loss from history

epochs_range = range(len(acc))             # Define range for epochs based on accuracy length

plt.figure(figsize=(10,8))                 # Set overall figure size for visualization

plt.subplot(1,2,1)                         # Create first subplot (1 row, 2 columns, position 1)
plt.plot(epochs_range, acc, label='Training Accuracy')       # Plot training accuracy
plt.plot(epochs_range, val_acc, label='Validation Accuracy') # Plot validation accuracy
plt.legend(loc='lower right')              # Place legend in lower-right corner
plt.title('Training vs Validation Accuracy') # Add title for accuracy plot

plt.subplot(1,2,2)                         # Create second subplot (1 row, 2 columns, position 2)
plt.plot(epochs_range, loss, label='Training Loss')         # Plot training loss
plt.plot(epochs_range, val_loss, label='Validation Loss')   # Plot validation loss
plt.legend(loc='upper right')              # Place legend in upper-right corner
plt.title('Training vs Validation Loss')   # Add title for loss plot

plt.show()                                 # Display the plots

<Figure size 1000x800 with 2 Axes>

In [44]:
loss, accuracy = model.evaluate(test_ds_eval)
print(f'Test accuracy is{accuracy:.4f}, Test loss is {loss:.4f}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 266ms/step - accuracy: 0.9296 - loss: 0.2713
Test accuracy is0.9141, Test loss is 0.2908


In [45]:
# Extract true labels from all batches in the test dataset
y_true = np.concatenate([y.numpy() for x, y in test_ds_eval], axis=0)  # Convert Tensor labels to NumPy array and concatenate them

# Get predictions as probabilities from the model
y_pred_probs = model.predict(test_ds_eval)  # Predict class probabilities for each sample in the test dataset

# Convert probabilities to predicted class indices
y_pred = np.argmax(y_pred_probs, axis=1)  # Select the class with the highest probability for each sample

# Compute the confusion matrix to evaluate classification performance
cm = confusion_matrix(y_true, y_pred)  # Generate confusion matrix comparing true labels to predicted labels

# Print metrics to assess model performance
print(cm)  # Display confusion matrix
print(classification_report(y_true, y_pred))  # Print precision, recall, and F1-score for each class 
print(classification_report(y_true, y_pred, target_names=class_names))



8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 229ms/step
[[38  0  0  0  0  1]
 [ 0 45  2  0  2  0]
 [ 0  1 47  0  0  0]
 [ 3  1  2 54  1  1]
 [ 1  2  3  0 37  1]
 [ 1  0  0  0  0 13]]
              precision    recall  f1-score   support

           0       0.88      0.97      0.93        39
           1       0.92      0.92      0.92        49
           2       0.87      0.98      0.92        48
           3       1.00      0.87      0.93        62
           4       0.93      0.84      0.88        44
           5       0.81      0.93      0.87        14

    accuracy                           0.91       256
   macro avg       0.90      0.92      0.91       256
weighted avg       0.92      0.91      0.91       256

              precision    recall  f1-score   support

   cardboard       0.88      0.97      0.93        39
       glass       0.92      0.92      0.92        49
       metal       0.87      0.98      0.92        48
       paper       1.00      0.87      0.93        62
     plastic       0

In [46]:
plt.figure(figsize=(10,8))  # Set figure size for better visualization

sns.heatmap(cm, annot=True, fmt='d',  # Create heatmap using confusion matrix
            xticklabels=class_names,  # Set class names for x-axis (predicted labels)
            yticklabels=class_names,  # Set class names for y-axis (true labels)
            cmap='Blues')  # Use a blue colormap for better contrast

plt.xlabel('Predicted')  # Label x-axis as Predicted classes
plt.ylabel('True')  # Label y-axis as True classes
plt.title('Confusion Matrix')  # Add title to the heatmap
plt.show()  # Display the plot

<Figure size 1000x800 with 2 Axes>

In [47]:
# Extract class names from the training dataset
class_names = train_ds.class_names  

# Take one batch of images and labels from the test dataset for evaluation
for images, labels in test_ds_eval.take(1):  

    # Generate predictions for the batch of images
    predictions = model.predict(images)  

    # Get the predicted class index for each image
    pred_labels = tf.argmax(predictions, axis=1)  

    # Loop through the first 8 images in the batch
    for i in range(10):  
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert and display image
        plt.title(f"True: {class_names[labels[i]]}, Pred: {class_names[pred_labels[i]]}")  # Show actual and predicted class
        plt.axis("off")  # Hide axes for better visualization
        plt.show()  # Display the image with title

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [48]:
# Save model in Keras format with architecture, weights, and training configuration
model.save('Effiicientnetv2b2.keras')

# Load your Keras model
model = tf.keras.models.load_model('Effiicientnetv2b2.keras')

In [49]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

In [50]:
def classify_image(img):  
    # Resize image to 124x124 pixels (Note: Comment says 128x128, but code resizes to 124x124)
    img = img.resize((124, 124))  
    
    # Convert image to a NumPy array with float32 dtype
    img_array = np.array(img, dtype=np.float32)  
    img_array = preprocess_input(img_array)
    
    # Expand dimensions to match model input shape (adds a batch dimension)
    img_array = np.expand_dims(img_array, axis=0)  
    
    # Make a prediction using the trained model
    prediction = model.predict(img_array)  
    
    # Get the index of the highest predicted probability
    predicted_class_index = np.argmax(prediction)  
    
    # Map the predicted index to its corresponding class name
    predicted_class_name = class_names[predicted_class_index]  
    
    # Extract confidence score (probability of the predicted class)
    confidence = prediction[0][predicted_class_index]  
    
    # Return formatted prediction result with confidence score
    return f"Predicted: {predicted_class_name} (Confidence: {confidence:.2f})"  

In [51]:
import tensorflow as tf
import cv2

def get_gradcam(model, img_array, class_index, last_conv_layer_name="top_activation"):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = heatmap.numpy()

    # Resize heatmap to original image size
    heatmap = cv2.resize(heatmap, (img_array.shape[2], img_array.shape[1]))
    heatmap = np.uint8(255 * heatmap)

    # Apply colormap
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    return heatmap


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from gtts import gTTS
import tempfile
import os

# 📦 Disposal tips
tips = {
    "plastic": "Rinse before recycling. Avoid black plastic.",
    "glass": "Remove lids. Don’t break it.",
    "metal": "Crush cans to save space.",
    "paper": "Keep it dry. No oil-soaked paper.",
    "cardboard": "Flatten it. Avoid food stains.",
    "trash": "Dispose responsibly. Not recyclable."
}

# Preprocess function
def preprocess_image(img):
    img = img.resize(image_size)  # example: (224, 224)
    img = np.array(img)
    return np.expand_dims(img, axis=0)

# Main prediction function
def classify_image(input_img):
    img = preprocess_image(input_img)
    preds = model.predict(img)[0]  # prediction vector

    # Top 3 predictions
    top3_indices = preds.argsort()[-3:][::-1]
    top3 = [(class_names[i], preds[i]) for i in top3_indices]

    result_text = "🔍 Top 3 Predictions:\n"
    for name, score in top3:
        result_text += f"{name}: {score:.2%}\n"

    # Recyclable or trash
    recyclable = {"paper", "plastic", "glass", "metal", "cardboard"}
    top_label = top3[0][0]
    is_recyclable = "♻️ Recyclable" if top_label in recyclable else "🚯 Trash"
    result_text += f"\nPrediction Type: {is_recyclable}"

    # Tip
    tip = tips[top_label]
    result_text += f"\n\n🧠 Tip: {tip}"

    # Voice output
    voice_msg = f"This image is classified as {top_label}. It is {is_recyclable.replace('♻️ ', '').replace('🚯 ', '')}."
    tts = gTTS(text=voice_msg)
    temp_audio_path = os.path.join(tempfile.gettempdir(), "prediction.mp3")
    tts.save(temp_audio_path)

    # Plot confidence
    fig, ax = plt.subplots()
    ax.bar(class_names, preds, color="skyblue")
    ax.set_title("Prediction Confidence")
    ax.set_ylabel("Confidence")
    ax.set_ylim([0, 1])
    plt.xticks(rotation=45)

    return result_text, fig, temp_audio_path

# Gradio interface with audio
gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil", label="Upload or Capture an Image"),
    outputs=[
        "text", 
        gr.Plot(label="Class Confidence"), 
        gr.Audio(label="Prediction Voice", autoplay=True)

    ],
    title="🗑️ Garbage Classifier",
    description="Upload an image of waste and let the model predict the category: cardboard, glass, metal, paper, plastic, or trash."
).launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
